# Example Notebook
- Connecting and Configuring
- Setting and Getting Parameters
- Image Acquisition
- Stage Movement
- FIB Milling
- Segmentation and Feature Detection
- Some Examples

## Connecting


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, movement
from fibsem.structures import BeamType, Point, FibsemImage, FibsemStagePosition

import numpy as np
import matplotlib.pyplot as plt
import os

# the default configuration is found in fibsem/config/user-configurations.yaml.
# if you want to use a different configuration, you can specify it there, in the ui, 
# or as an argument to setup_session
# the configuration is stored in microscope.system and returned from setup_session as settings.

# connect to microscope
CONFIG_PATH = "../fibsem/config/microscope-configuration.yaml"
microscope, settings = utils.setup_session(config_path=CONFIG_PATH)


## Setting and Getting Parameters

In [ ]:
# set
microscope.set("working_distance", 4e-3, beam_type=BeamType.ELECTRON)
microscope.set("beam_current", 1e-9, beam_type=BeamType.ION)

# get 
wd = microscope.get("working_distance", beam_type=BeamType.ELECTRON)
print(f"Working distance: {wd} m")

bc = microscope.get("beam_current", beam_type=BeamType.ION)
print(f"Ion beam current: {bc} A")

beam_settings = microscope.get_beam_settings(beam_type=BeamType.ELECTRON)
print(f"Beam settings: {beam_settings}")

In [ ]:
# you can list available parameters with get_available_values. Note that not every value returns a list of available values.

fib_currents_available = microscope.get_available_values("current", beam_type=BeamType.ION)
print(f"Available FIB Currents: {fib_currents_available}")

fib_plasma_gases = microscope.get_available_values("plasma_gas", beam_type=BeamType.ION)
print(f"Available FIB Plasma Gases: {fib_plasma_gases}")

In [ ]:
# following is used for changing system settings. The defaults are loaded from the configuration.yaml file.

# is available?
print('FIB Plasma is available:', microscope.is_available("ion_plasma"))
# set available
microscope.set_available("ion_plasma", True)
print('FIB Plasma is available:', microscope.is_available("ion_plasma"))

# change plasma gas
# microscope.set("plasma_gas", "Xenon", beam_type=BeamType.ION) # uncomment this line, if you want to change the plasma gas (it will take ~10 mins)
print('Plasma gas:', microscope.get("plasma_gas", beam_type=BeamType.ION))

## Image Acquisition

In [ ]:
from fibsem import acquire
from fibsem.structures import ImageSettings

# image settings
image_settings = ImageSettings(
    resolution = [1536, 1024], 
    dwell_time=1.0e-6,
    hfw=80-6,
    beam_type=BeamType.ELECTRON,
    autocontrast=True,
)

# acquire image
image = acquire.new_image(microscope, image_settings)

In [ ]:
# acquire multiple images with different hfw (and saving them)
image_settings.beam_type = BeamType.ELECTRON
image_settings.autocontrast = False
image_settings.save = True
image_settings.path = os.path.join(os.getcwd(), "demo", "imaging")
os.makedirs(image_settings.path, exist_ok=True)

hfws = [80e-6, 150e-6, 400e-6, 900e-6]
for i, hfw in enumerate(hfws):

    image_settings.hfw = hfw
    image_settings.filename = f"hfws_{i:02d}"

    sem_image = acquire.new_image(microscope, image_settings)


In [ ]:
# loading images from file

import glob
from fibsem.structures import FibsemImage

filenames = sorted(glob.glob(os.path.join(image_settings.path, "hfws*.tif")))

# plot wiht subplot
fig, axes = plt.subplots(1, len(filenames), figsize=(15, 5))
for fname in filenames:

    image = FibsemImage.load(fname)
    ax = axes[filenames.index(fname)]
    ax.imshow(image.data, cmap="gray")
    ax.set_title(f"{image.metadata.image_settings.filename} - {image.metadata.image_settings.hfw*1e6:.0f} um")
    ax.axis("off")

# plotting 
plt.tight_layout()
plt.subplots_adjust(wspace=0.01)
plt.savefig(os.path.join(image_settings.path, "hfws.png"), dpi=300)
plt.show()

In [ ]:
# acquire images with both beams
image_settings.save = False
sem_image, fib_image = acquire.take_reference_images(microscope, image_settings)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(sem_image.data, cmap="gray")
ax[0].set_title("SEM")
ax[0].axis("off")
ax[1].imshow(fib_image.data, cmap="gray")
ax[1].set_title("FIB")
ax[1].axis("off")
plt.tight_layout()
plt.show()


## Stage Movement
Core stage movement functionality

In [ ]:
# helper for plotting images
def plot_images(sem_image0, fib_image0, sem_image1, fib_image1):
    # plot images
    fig, axes = plt.subplots(2, 2, figsize=(10, 7))
    axes[0][0].imshow(sem_image0.data, cmap="gray")
    axes[0][0].set_title("SEM (pre-movement)")
    # crosshair, centre, yellow
    axes[0][0].plot(sem_image0.data.shape[1]/2, sem_image0.data.shape[0]/2, "y+", ms=20)
    axes[0][1].imshow(fib_image0.data, cmap="gray")
    axes[0][1].set_title("FIB (pre-movement)")
    axes[0][1].plot(fib_image0.data.shape[1]/2, fib_image0.data.shape[0]/2, "y+", ms=20)

    axes[1][0].imshow(sem_image1.data, cmap="gray")
    axes[1][0].set_title("SEM (post-movement)")
    # crosshair, centre, yellow
    axes[1][0].plot(sem_image1.data.shape[1]/2, sem_image1.data.shape[0]/2, "y+", ms=20)
    axes[1][1].imshow(fib_image1.data, cmap="gray")
    axes[1][1].set_title("FIB (post-movement)")
    axes[1][1].plot(fib_image1.data.shape[1]/2, fib_image1.data.shape[0]/2, "y+", ms=20)

    # axes off
    for ax in axes.flatten():
        ax.axis("off")

    plt.subplots_adjust(hspace=0.07, wspace=0.01)
    plt.show()

## Move Stage (Relative and Absolute)
Moves the physical stage axes (x, y, z, r, t). For pre-tilted stage will result in losing coincidence and focus and you move around. 

In [ ]:
# image settings
image_settings.hfw = 150e-6
image_settings.save = True
image_settings.path = os.path.join(os.getcwd(), "demo", "movement")
os.makedirs(image_settings.path, exist_ok=True)

# acquire images with both beams
image_settings.filename = "pre-movement"
eb_image0, ib_image0 = acquire.take_reference_images(microscope, image_settings)

# move stage by 50um
move_position = FibsemStagePosition(x=50e-6)
microscope.move_stage_relative(move_position)

# acquire images with both beams
image_settings.filename = "post-movement"
eb_image1, ib_image1 = acquire.take_reference_images(microscope, image_settings)

# plot images
plot_images(eb_image0, ib_image0, eb_image1, ib_image1)


### Stable Movement
Moves the stage along the sample-plane, and maintains coincidence between the SEM and FIB. Works at any orientation. Assumes that the beams are already coincident, see vertical move for restoring coincidence. Requires a beam type because it also corrects for perspective between the sample-plane and imaging-plane.

In [ ]:
image_settings.resolution = [1536, 1024]
image_settings.hfw = 150e-6
image_settings.filename = "pre-stable-movement"
sem_image0, fib_image0 = acquire.take_reference_images(microscope, image_settings)

# move along sample plane by 20um in x and y
microscope.stable_move(dx=20e-6, dy=20e-6, beam_type=BeamType.ELECTRON)

image_settings.filename = "post-stable-movement"
sem_image1, fib_image1 = acquire.take_reference_images(microscope, image_settings)

# plot images
plot_images(sem_image0, fib_image0, sem_image1, fib_image1)


### Vertical Movement
Moves the stage vertical in the chamber. Used to restored coincidence between the SEM and FIB. The feature should be centred first in the SEM, then the vertical move applied based on the feature position in the FIB. After vertical movement, the feature should stay cented in the SEM, and also be centred in the FIB.

In [ ]:
image_settings.resolution = [1536, 1024]
image_settings.hfw = 150e-6
image_settings.filename = "pre-vertical-movement"
sem_image0, fib_image0 = acquire.take_reference_images(microscope, image_settings)

# move vertical by 20um
microscope.vertical_move(dy=20e-6)

image_settings.filename = "post-vertical-movement"
sem_image1, fib_image1 = acquire.take_reference_images(microscope, image_settings)

# plot images
plot_images(sem_image0, fib_image0, sem_image1, fib_image1)


## FIB Milling

In [ ]:
from fibsem import milling
from fibsem.milling import FibsemMillingStage, MillingAlignment
from fibsem.structures import FibsemMillingSettings, FibsemRectangleSettings, FibsemCircleSettings, FibsemLineSettings, FibsemImage, Point
from fibsem.milling.patterning.patterns2 import RectanglePattern, TrenchPattern, CirclePattern, LinePattern
from fibsem.milling.patterning.plotting import draw_milling_patterns

# Shapes
# the fib can only draw different individual shapes, such as rectangles, circles, and lines.
# the settings for these shapes are defined in the respective classes (FibsemRectangleSettings, FibsemCircleSettings, FibsemLineSettings, ...)
rect_settings = FibsemRectangleSettings(width=10e-6, height=10e-6, depth=1e-6, centre_x=0, centre_y=0)

# Patterns
# typically we don't want to interact with the shapes directly, but use the higher level 'patterns', RectanglePattern, CirclePattern, LinePattern, ...
# there are more complex patterns available, such as TrenchPattern, which is a combination of rectangles.
# patterns have more advanced parameters that can be used to adjust how that pattern is drawn/millied, e.g. passes, cross_section.
rectangle_pattern = RectanglePattern(
    width=10e-6,
    height=10e-6,
    depth=1e-6,
    point=Point(0, 0),
)

trench_pattern = TrenchPattern(
    width=10e-6,
    upper_trench_height=5e-6,
    lower_trench_height=5e-6,
    spacing=3e-6,
    depth=1e-6,
    point=Point(0, 0),
)

# calling .define() on a pattern will return a list of shapes that can be used to draw the pattern
print(f"Shapes that make up the rectangle pattern: {rectangle_pattern.define()}")   # 1 rectangle
print(f"Shapes that make up the trench pattern: {trench_pattern.define()}")         # 2 rectangles


# MillingSettings
# milling settings define the beam settings and parameters for the milling process
milling_settings = FibsemMillingSettings(
    milling_current=1e-9,
    milling_voltage=30e3,
    hfw=80e-6,
    application_file="Si",
    patterning_mode="Serial",
)

# Alignment
# the alignment is used to align the between the initial imaging current and milling current (disable it for now)
milling_alignment = MillingAlignment(
    enabled=False,
)

# Milling Stage
# all these parameters are joined together in the milling stage, which is used to run the milling process
milling_stage = FibsemMillingStage(
    name="Milling Stage",
    milling=milling_settings,
    pattern = rectangle_pattern,
    alignment=milling_alignment,
)

# Utilities
# there are some utils availble for generating patterns and plotting them
image = FibsemImage.generate_blank_image(hfw=milling_stage.milling.hfw)
fig = draw_milling_patterns(image, [milling_stage])


In [ ]:
## deconstructing the milling process
# 1. setup milling: (align, set beam settings, ...)
milling.setup_milling(microscope, milling_stage)

# 2. draw patterns (shapes)
milling.draw_patterns(microscope, milling_stage.pattern.define())

# 3. run milling
milling.run_milling(microscope, milling_stage.milling.milling_current, milling_stage.milling.milling_voltage)

# 4. finish milling (restore imaging beam settings, clear shapes, ...)
milling.finish_milling(microscope)

In [ ]:
# it is recommended to use the mill_stages function, which will run all these steps for you.
# it will also take care of logging, imaging acquisition and strategies (which are not described here).
milling.mill_stages(microscope, milling_stage)

In [ ]:
# you can mill multiple stages in a row by passing a list of milling stages to mill_stages:
# this example shows how to load milling stages from a protcol file and join them together
from fibsem.utils import load_protocol
from fibsem.milling import get_milling_stages, mill_stages
from fibsem.milling.patterning.plotting import draw_milling_patterns
from fibsem.structures import FibsemImage
import matplotlib.pyplot as plt

# NOTE: you will need to change this path to the autolamella protocol path on your system
PROTOCOL_PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/protocol/protocol-on-grid.yaml"
protocol = load_protocol(PROTOCOL_PATH)

# load each set of milling stages
rough_milling_stages = get_milling_stages("mill_rough", protocol=protocol["milling"])
polishing_milling_stages = get_milling_stages("mill_polishing", protocol=protocol["milling"])
microexpansion_stages = get_milling_stages("microexpansion", protocol=protocol["milling"])
fiducial_stages = get_milling_stages("fiducial", protocol=protocol["milling"])

milling_stages = rough_milling_stages + polishing_milling_stages + microexpansion_stages + fiducial_stages

image = FibsemImage.generate_blank_image(hfw=milling_stages[0].milling.hfw)
fig = draw_milling_patterns(image, milling_stages)
plt.show()


In [ ]:
# mill all stages
mill_stages(microscope, milling_stages)

## Segmentation and Feature Detection

Example to load a machine learning model, segment and detect specific features, and plot the output on the image.

In [ ]:
# NOTE: this will need the additional ml dependencies to work: pip install fibsem[ml]

from fibsem.detection import detection
from fibsem.segmentation import model
from fibsem.detection import detection
from fibsem.detection.detection import DetectedFeatures
from fibsem.segmentation import model as fibsem_model
from fibsem.segmentation.model import load_model
from fibsem.structures import (
    BeamType,
    FibsemImage,
    Point,
)

# model parameters (downloaded from huggingface/patrickcleeve/autolamella)
checkpoint = "autolamella-mega-20240107.pt"

# load model
model = load_model(checkpoint=checkpoint)

# load image
image = FibsemImage.load("../fibsem/detection/test_image.tif")

# detect features
features = [detection.NeedleTip(), detection.LamellaCentre()] 
det = detection.detect_features(image=image.data, model=model, 
                                features=features, pixelsize=25e-9)

# plot detection
detection.plot_detection(det)

## Tiled Imaging Examples
Collect a tile set using the SEM. First moves to the SEM orientation (sample perpendicular to the SEM), and then acquires a 3x3 tileset. Images are saved in the demo/tile direction, and the cell afterwards will load them back and 'stitch' them together. Real tiled acquisition is in fibsem/imaging/tiled.

In [ ]:
from fibsem import utils, acquire
from fibsem.structures import FibsemStagePosition
import os
import matplotlib.pyplot as plt

# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Demo", ip_address="localhost")

# move to SEM orientation
microscope.move_flat_to_beam(BeamType.ELECTRON)

# image settings
image_settings = settings.image
image_settings.hfw = 80e-6
image_settings.resolution = [1024, 1024]
image_settings.beam_type = BeamType.ELECTRON
image_settings.save = True 
image_settings.path = os.path.join(os.getcwd(), "demo", "tile")
os.makedirs(image_settings.path, exist_ok=True)

# tile settings
dx, dy = image_settings.hfw, image_settings.hfw
nrows, ncols = 3, 3

# tile
initial_position = microscope.get_stage_position()
for i in range(nrows):

    # restore position
    microscope.move_stage_absolute(initial_position)
    # stable movement dy
    microscope.stable_move(dx=0, dy=dy*i, beam_type=BeamType.ELECTRON)

    for j in range(ncols):
    
        # stable movement dx
        microscope.stable_move(dx=dx, dy=0, beam_type=BeamType.ELECTRON)
        # acquire images with both beams
        image_settings.filename = f"tile_{i:03d}_{j:03d}"
        ib_image = acquire.new_image(microscope, image_settings)

In [ ]:
# plot tiles
import glob
filenames = sorted(glob.glob(os.path.join(image_settings.path, "tile*.tif")))


fig, axes = plt.subplots(nrows, ncols, figsize=(10, 10))
for i, fname in enumerate(filenames):

    image = FibsemImage.load(fname)
    ax = axes[i//ncols][i%ncols]
    ax.imshow(image.data, cmap="gray")
    ax.axis("off")

plt.tight_layout()
plt.subplots_adjust(hspace=0.01,wspace=0.01)
plt.savefig(os.path.join(image_settings.path, "tiles.png"), dpi=300)
plt.show()